In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from collections import defaultdict, namedtuple
import os
import csv

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My\ Drive/

from PIL import Image

import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(229)

import pandas as pd

from tqdm import tqdm

Mounted at /content/drive
/content/drive/My Drive


# **SETUP:**


## Set up ImageDataset class to get image input

In [5]:
class ImageDataset(Dataset):
    def __init__(self, img_dir, transform, data_dir):
        self.img_dir = img_dir
        self.transform = transform
        self.data_dir = data_dir
        self.data = []

        self.imgs = []

        self.one_hot_labels = []

        self.data = pd.read_csv(self.data_dir, names=['filename', 'label'], sep=' ')

        for idx, d in tqdm(self.data.iterrows()):
          filename, label = d['filename'], d['label']

          try:
            image = Image.open(os.path.join(self.img_dir,filename)).convert("RGB")
          except FileNotFoundError:
            self.data = self.data.drop(idx)
          else:
            self.imgs.append((filename, image))
            one_hot_label = np.array([int(i == label) for i in range(30)])
            self.one_hot_labels.append(one_hot_label)
                        
    def __len__(self):
        """Return the length of the dataset—how many images there are, total."""
        return len(self.imgs)

    def get_one_hot_labels(self):
      return self.one_hot_labels

    def get_one_hot_label(self, idx):
      return self.one_hot_labels[idx]

    def get_data(self, idx):
      return self.data.iloc[idx]
        
    def get_image(self, idx):
        """Returns the image at this index."""
        return self.imgs[idx]

    def get_label(self, idx):
        """Given an index, return the ground truth label for that index."""
        return self.one_hot_labels[idx]
        # return self.data[idx]['label']
    
    def __getitem__(self, idx):
        """Return the image and label at a given index"""
        tensor_image = self.transform(self.get_image(idx)[1])
        label = self.get_label(idx)

        return tensor_image, label
    
    def display(self, idx):
        """Displays the image at a given index"""
        display(self.get_image(idx)[1])



## Create training set

Note: change train_set variable to dset for this cell and afterwards for next run


In [ ]:
img_dir = '224x224'
train_data_dir = 'bookcover30-labels-train.csv'
train_set = ImageDataset(img_dir, ToTensor(), train_data_dir)

0it [00:00, ?it/s]

In [11]:
# TRAINING DATASET
ix = 0
train_set.display(ix)
print(train_set.get_data(ix))

[]


IndexError: ignored

In [ ]:
# PREDICT USING FIRST LABEL

predicted_label = train_set.get_label(0)

test_data_dir = 'bookcover30-labels-test.csv'
test_set = ImageDataset(img_dir, ToTensor(), test_data_dir)

## Create dataloader for training and validation set

In [ ]:
validation_split = 0.10
dataset_size = len(dset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
train_dataloader = DataLoader(dset, batch_size=32, sampler=train_sampler)
valid_dataloader = DataLoader(dset, batch_size=32, sampler=valid_sampler)

**BASELINE MODEL:**

**PRELIMINARY EXPERIMENT:**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3,10,5)    # 224 x 224 x  3 > 220 x 220 x 10
        self.pool1 = nn.MaxPool2d(2,2)    # 220 x 220 x 10 > 110 x 110 x 10
        self.conv2 = nn.Conv2d(10,20,5)   # 110 x 110 x 10 > 106 x 106 x 20
        self.pool2 = nn.MaxPool2d(2,2)    # 106 x 106 x 20 >  53 x  53 x 20

        self.conv3 = nn.Conv2d(20,30,6)   #  53 x  61 x 20 >  56 x  56 x 30
        self.pool3 = nn.MaxPool2d(2,2)    #  56 x  56 x 30 >  28 x  28 x 30
        self.conv4 = nn.Conv2d(30,30,5)   #  28 x  28 x 30 >  24 x  24 x 30
        self.pool4 = nn.MaxPool2d(2,2)    #  24 x  24 x 30 >  12 x  12 x 30 (=4320)

        self.fc1 = nn.Linear(4320,700)    #      4320     >      700
        self.fc2 = nn.Linear(700,70)      #       700     >       70
        self.fc3 = nn.Linear(70,1)        #        70     >        1

        # define your layers

    def forward(self, x):
        # run your data through the layers
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, 4320)
        x = self.fc2(F.relu(self.fc1(x)))
        x = self.fc3(F.relu(x))
        return x.squeeze()